<a href="https://www.kaggle.com/code/malavikashamesh/customer-segmentation?scriptVersionId=165343012" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-data/data.csv


In [2]:
# Loading modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns

In [3]:
# read the datafile
data = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding="ISO-8859-1", dtype={'CustomerID': str,'InvoiceID': str})
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [4]:
# Data Preprocessing

# Drop rows with missing values
data.dropna(inplace=True)

# Convert 'InvoiceDate' column to datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Calculate total purchase amount for each transaction
data['TotalAmount'] = data['Quantity'] * data['UnitPrice']

In [5]:
# Encode categorical variable 'Country' using one-hot encoding
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(data[['Country']])
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(['Country']))

# Concatenate the original dataframe with the one-hot encoded dataframe
data_one_hot_encoded = pd.concat([data, one_hot_encoded_df], axis=1)

# Encode categorical variable 'Country' using label encoding
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(data['Country'])
data['Country_LabelEncoded'] = label_encoded

# Normalize numerical features 'Quantity' and 'UnitPrice'
scaler = StandardScaler()
data[['Quantity', 'UnitPrice']] = scaler.fit_transform(data[['Quantity', 'UnitPrice']])

# Display the updated dataframe
print(data.head())

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER -0.024373   
1    536365     71053                  WHITE METAL LANTERN -0.024373   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER -0.016331   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE -0.024373   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART. -0.024373   

          InvoiceDate  UnitPrice CustomerID         Country  TotalAmount  \
0 2010-12-01 08:26:00  -0.013135      17850  United Kingdom        15.30   
1 2010-12-01 08:26:00  -0.001017      17850  United Kingdom        20.34   
2 2010-12-01 08:26:00  -0.010250      17850  United Kingdom        22.00   
3 2010-12-01 08:26:00  -0.001017      17850  United Kingdom        20.34   
4 2010-12-01 08:26:00  -0.001017      17850  United Kingdom        20.34   

   Country_LabelEncoded  
0                    35  
1                    35  
2                    35  
3     

In [6]:
# Calculate total purchase amount for each customer
total_purchase_amount = data.groupby('CustomerID')['TotalAmount'].sum().reset_index()
total_purchase_amount.rename(columns={'TotalAmount': 'TotalPurchaseAmount'}, inplace=True)

# Calculate frequency of purchases for each customer
frequency_of_purchases = data.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
frequency_of_purchases.rename(columns={'InvoiceNo': 'FrequencyOfPurchases'}, inplace=True)

# Calculate recency of purchases for each customer
recency_of_purchases = data.groupby('CustomerID')['InvoiceDate'].max().reset_index()
current_date = pd.to_datetime('now')
recency_of_purchases['RecencyOfPurchases'] = (current_date - recency_of_purchases['InvoiceDate']).dt.days
recency_of_purchases.drop(columns=['InvoiceDate'], inplace=True)

# Merge all extracted features into a single DataFrame
customer_features = total_purchase_amount.merge(frequency_of_purchases, on='CustomerID').merge(recency_of_purchases, on='CustomerID')

# Display the extracted features
print(customer_features.head())

  CustomerID  TotalPurchaseAmount  FrequencyOfPurchases  RecencyOfPurchases
0      12346                 0.00                     2                4793
1      12347              4310.00                     7                4470
2      12348              1797.24                     4                4543
3      12349              1757.55                     1                4486
4      12350               334.40                     1                4778
